Download data van ENTSO-E

In [2]:
import pandas as pd
from entsoe import EntsoePandasClient
import time

# Insert your API key here once you get it
API_KEY = '82aa28d4-59f3-4e3a-b144-6659aa9415b5'

# Initialize ENTSO-E client
client = EntsoePandasClient(api_key=API_KEY)

# Define parameters
country_code = 'NL'
neighboring_countries = ['BE', 'DE', 'GB', 'DK', 'NO']  # Pas dit aan op basis van de relevante buren
years = [2022, 2023, 2024]  # List of years to fetch

# Data storage
all_data = []

# Function to fetch data with retries
def fetch_with_retries(fetch_func, *args, retries=3, delay=5, **kwargs):
    for attempt in range(retries):
        try:
            return fetch_func(*args, **kwargs)
        except Exception as e:
            print(f"Error: {e}, retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception(f"Failed to fetch data after {retries} attempts")

# Loop through each year and fetch data separately
for year in years:
    start = pd.Timestamp(f'{year}-01-01', tz='Europe/Amsterdam')
    end = pd.Timestamp(f'{year+1}-01-01', tz='Europe/Amsterdam')  # Exclusive end

    print(f"Fetching load data for {year}...")
    yearly_load = fetch_with_retries(client.query_load, country_code, start=start, end=end).squeeze()  # Convert to 1D Series

    print(f"Fetching load forecast for {year}...")
    yearly_load_forecast = fetch_with_retries(client.query_load_forecast, country_code, start=start, end=end).squeeze()  # Convert to 1D Series

    print(f"Fetching price data for {year}...")
    yearly_price = fetch_with_retries(client.query_day_ahead_prices, country_code, start=start, end=end).squeeze()  # Convert to 1D Series

    # Fetch cross-border flows
    flow_data = {}
    for neighbor in neighboring_countries:
        print(f"Fetching cross-border flow from {neighbor} to {country_code} for {year}...")
        yearly_flow_to = fetch_with_retries(client.query_crossborder_flows, country_code_from=neighbor, 
                                            country_code_to=country_code, start=start, end=end).squeeze()  # Convert to 1D Series
        flow_data[f'Flow_{neighbor}_to_{country_code}'] = yearly_flow_to

        print(f"Fetching cross-border flow from {country_code} to {neighbor} for {year}...")
        yearly_flow_from = fetch_with_retries(client.query_crossborder_flows, country_code_from=country_code, 
                                              country_code_to=neighbor, start=start, end=end).squeeze()  # Convert to 1D Series
        flow_data[f'Flow_{country_code}_to_{neighbor}'] = yearly_flow_from

    # Merge all data
    if not yearly_load.empty and not yearly_price.empty:
        df = pd.DataFrame({'Load': yearly_load, 'Price': yearly_price})
        for col_name, flow_series in flow_data.items():
            if not flow_series.empty:
                df[col_name] = flow_series

        # Store yearly data
        all_data.append(df)
    else:
        print(f"No data for year {year}")

# Concatenate all years into one DataFrame if there is data
if all_data:
    final_data = pd.concat(all_data)
    # Save to CSV
    final_data.to_csv('electricity_data_nl_2022_2024.csv')
    print("Data saved successfully!")
else:
    print("No data to save.")

Fetching load data for 2022...
Fetching load forecast for 2022...
Fetching price data for 2022...


Connection Error, retrying in 0 seconds


Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), retrying in 5 seconds...
Fetching cross-border flow from BE to NL for 2022...
Fetching cross-border flow from NL to BE for 2022...
Fetching cross-border flow from DE to NL for 2022...
Fetching cross-border flow from NL to DE for 2022...
Fetching cross-border flow from GB to NL for 2022...
Fetching cross-border flow from NL to GB for 2022...
Fetching cross-border flow from DK to NL for 2022...
Fetching cross-border flow from NL to DK for 2022...
Fetching cross-border flow from NO to NL for 2022...
Fetching cross-border flow from NL to NO for 2022...
Fetching load data for 2023...
Fetching load forecast for 2023...
Fetching price data for 2023...


Connection Error, retrying in 0 seconds


Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), retrying in 5 seconds...
Fetching cross-border flow from BE to NL for 2023...
Fetching cross-border flow from NL to BE for 2023...
Fetching cross-border flow from DE to NL for 2023...
Fetching cross-border flow from NL to DE for 2023...
Fetching cross-border flow from GB to NL for 2023...
Fetching cross-border flow from NL to GB for 2023...
Fetching cross-border flow from DK to NL for 2023...
Fetching cross-border flow from NL to DK for 2023...
Fetching cross-border flow from NO to NL for 2023...
Fetching cross-border flow from NL to NO for 2023...
Fetching load data for 2024...
Fetching load forecast for 2024...
Fetching price data for 2024...


Connection Error, retrying in 0 seconds


Fetching cross-border flow from BE to NL for 2024...
Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), retrying in 5 seconds...
Fetching cross-border flow from NL to BE for 2024...
Fetching cross-border flow from DE to NL for 2024...
Fetching cross-border flow from NL to DE for 2024...
Fetching cross-border flow from GB to NL for 2024...
Fetching cross-border flow from NL to GB for 2024...
Fetching cross-border flow from DK to NL for 2024...
Fetching cross-border flow from NL to DK for 2024...
Fetching cross-border flow from NO to NL for 2024...
Fetching cross-border flow from NL to NO for 2024...
Data saved successfully!


Read excel file + data per uur

In [ ]:
import pandas as pd
from entsoe import EntsoePandasClient
import time

try:
    # Load the existing CSV file
    df = pd.read_csv('electricity_data_nl_2022_2024.csv', index_col=0, parse_dates=True)
    print("CSV file loaded successfully!")

    # Ensure the index is datetime with UTC
    df.index = pd.to_datetime(df.index, utc=True)
    print("Index converted to datetime with UTC!")

    # Shift timestamps by 1 hour to calculate the mean for the past hour
    df.index = df.index - pd.Timedelta(hours=1)

    # Resample the data to hourly frequency and calculate the mean for each hour
    df_hourly = df.resample('h').mean()
    print("Data resampled to hourly frequency based on the past hour!")

    # Save the resampled data to a new CSV file
    df_hourly.to_csv('electricity_data_nl_2022_2024_hourly.csv')
    print("Hourly data with flow saved successfully!")

except FileNotFoundError:
    print("Error: The file 'electricity_data_nl_2022_2024.csv' does not exist.")
except pd.errors.EmptyDataError:
    print("Error: The file 'electricity_data_nl_2022_2024.csv' is empty.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

CSV file loaded successfully!
Index converted to datetime with UTC!
Data resampled to hourly frequency based on the past hour!
Hourly data with flow saved successfully!


In [ ]:
#Nieuwe variabelen flow toevoegen (per land en totaal)

In [1]:
neighboring_countries = ['BE', 'DE', 'GB', 'DK', 'NO']

for neighbor in neighboring_countries:
    df_hourly[f'Flow_{neighbor}'] = df_hourly[f'Flow_{neighbor}_to_NL'] - df_hourly[f'Flow_NL_to_{neighbor}']

df_hourly['Total_Flow'] = df_hourly['Flow_BE'] + df_hourly['Flow_DE'] + df_hourly['Flow_GB'] + df_hourly['Flow_DK'] + df_hourly['Flow_DK']
df_hourly.to_csv('electricity_data_nl_2022_2024_hourly_flow.csv')


NameError: name 'df_hourly' is not defined